In [1]:
import torch
import cv2
import os
import sys
import threading
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk

# Fix PosixPath issue on Windows
os.environ['PYTHONPATH'] = os.getcwd()
if sys.platform == 'win32':
    import pathlib
    pathlib.PosixPath = pathlib.WindowsPath

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

running = False  # Global variable to control real-time detection
confidence_threshold = 0.25
detected_image = None  # Global variable to store the detected image

def start_detection():
    global cap, running
    running = True
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        messagebox.showerror("Erreur", "Impossible d'ouvrir la webcam")
        return

    while running:
        ret, frame = cap.read()
        if not ret:
            messagebox.showerror("Erreur", "Échec de la capture d'image")
            break
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(frame_rgb)
        results.xyxy[0] = results.xyxy[0][results.xyxy[0][:, 4] > confidence_threshold]
        rendered_frame = results.render()[0]
        rendered_frame = cv2.cvtColor(rendered_frame, cv2.COLOR_RGB2BGR)

        cv2.imshow("Détection en temps réel - YOLOv5", rendered_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_detection()
            break

def stop_detection():
    global running
    running = False
    if 'cap' in globals() and cap.isOpened():
        cap.release()
    cv2.destroyAllWindows()

def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Images", "*.jpg;*.png;*.jpeg")])
    if not file_path:
        return
    img = cv2.imread(file_path)
    display_image(img, label_original)
    process_and_display(img)

def capture_image():
    global detected_image
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Erreur", "Impossible d'ouvrir la webcam")
        return
    ret, frame = cap.read()
    cap.release()
    if not ret:
        messagebox.showerror("Erreur", "Échec de la capture d'image")
        return
    display_image(frame, label_original)
    process_and_display(frame)

def display_image(img, label):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    img_pil.thumbnail((300, 300))
    img_tk = ImageTk.PhotoImage(img_pil)
    label.config(image=img_tk)
    label.image = img_tk

def process_and_display(img):
    global detected_image
    frame_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model(frame_rgb)
    results.xyxy[0] = results.xyxy[0][results.xyxy[0][:, 4] > confidence_threshold]
    rendered_frame = results.render()[0]
    rendered_frame = cv2.cvtColor(rendered_frame, cv2.COLOR_RGB2BGR)
    detected_image = rendered_frame
    display_image(rendered_frame, label_result)
    messagebox.showinfo("Succès", "Détection terminée !")

def save_detected_image():
    global detected_image
    if detected_image is not None:
        file_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
        if file_path:
            cv2.imwrite(file_path, detected_image)
            messagebox.showinfo("Succès", "Image enregistrée avec succès !")
    else:
        messagebox.showerror("Erreur", "Aucune image détectée à enregistrer !")

def clear_interface():
    label_original.config(image='')
    label_result.config(image='')
    label_original.image = None
    label_result.image = None

def update_confidence():
    global confidence_threshold
    try:
        confidence_threshold = float(confidence_entry.get())
        messagebox.showinfo("Succès", f"Seuil de confiance défini à {confidence_threshold}")
    except ValueError:
        messagebox.showerror("Erreur", "Valeur invalide. Veuillez entrer un nombre entre 0 et 1.")

# Interface graphique
root = tk.Tk()
root.title("Détection d'Objets YOLOv5 - ICOSNET")
root.geometry("750x500")
root.configure(bg="#002147")

frame_top = tk.Frame(root, bg="#002147")
frame_top.pack()

label = tk.Label(frame_top, text="Choisissez une option :", font=("Arial", 14), fg="white", bg="#002147")
label.pack(pady=10)

confidence_label = tk.Label(frame_top, text="Seuil de confiance (0-1) :", font=("Arial", 12), fg="white", bg="#002147")
confidence_label.pack(pady=5)

confidence_entry = tk.Entry(frame_top, font=("Arial", 12))
confidence_entry.insert(0, str(confidence_threshold))
confidence_entry.pack(pady=5)

submit_confidence = tk.Button(frame_top, text="Valider", command=update_confidence, font=("Arial", 12), bg="#0066cc", fg="white")
submit_confidence.pack(pady=5)

btn_frame = tk.Frame(root, bg="#002147")
btn_frame.pack()

buttons = [
    ("Démarrer la détection", lambda: threading.Thread(target=start_detection).start(), "#004080"),
    ("Arrêter", stop_detection, "#cc0000"),
    ("Charger une image", upload_image, "#00509e"),
    ("Capturer une image", capture_image, "#008000"),
    ("Enregistrer l'image détectée", save_detected_image, "#003399"),
    ("Effacer", clear_interface, "#808080")
]

for i, (text, command, color) in enumerate(buttons):
    btn = tk.Button(btn_frame, text=text, command=command, font=("Arial", 12), bg=color, fg="white")
    btn.grid(row=0, column=i, padx=5, pady=5)

image_frame = tk.Frame(root, bg="#002147")
image_frame.pack()

tk.Label(image_frame, text="Image Originale", font=("Arial", 12), fg="white", bg="#002147").grid(row=0, column=0)
tk.Label(image_frame, text="Image après Détection", font=("Arial", 12), fg="white", bg="#002147").grid(row=0, column=1)

label_original = tk.Label(image_frame, bg="#002147")
label_original.grid(row=1, column=0, padx=20, pady=5)
label_result = tk.Label(image_frame, bg="#002147")
label_result.grid(row=1, column=1, padx=20, pady=5)

root.mainloop()


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\PICOS/.cache\torch\hub\master.zip
YOLOv5  2025-2-28 Python-3.12.4 torch-2.6.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
C:\Users\PICOS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\PICOS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\PICOS/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\PICOS/.cach